<a href="https://colab.research.google.com/github/WolfgangHou/brain_health_product_hk/blob/main/2024_05_16_%E5%A4%A7%E8%85%A6%E7%94%A2%E5%93%81%E5%88%86%E6%9E%90(%E4%BD%9C%E5%93%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資料匯入、清洗與轉換

In [1]:
# import related module
import requests
import pandas as pd
import jieba
import time
import random
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
file_name = '2024-05-07_hktvmall_brainHealthProduct'

df = pd.read_csv(f'/content/drive/My Drive/Colab Notebooks/{file_name}.csv', index_col=False)
df.shape

(2160, 13)

In [3]:
import re
import pandas as pd

#萃取產品的包裝規格

# 儲存量詞之tuple，規格單位採用盡可能多樣中文量詞
quantifiers = ('丸','克','毫升','ml','g','個', '隻', '顆', '枚', '粒', '串', '張', '盒', '片', '罐', '瓶', '杯', '碗', '包', '塊', '支', '棵', '根', '條', '樽', '袋', '箱', '蒲', '副', '疋', '尾', '匹', '門', '口', '家', '頭', '位', '把', '台', '部', '棟', '間', '輛', '架', '輪', '座', '艘', '列', '車', '線', '號', '階', '層', '篇', '本', '冊', '輯', '份', '席', '曲', '卷', '步', '拍', '聲', '刻', '次', '回', '場', '波', '梭', '節', '袖', '堆', '塊', '團', '箇', '輩', '雙', '峰', '對', '班', '旅', '勺', '齣', '滴', '招', '朶', '缽', '帖', '彎', '攤', '套', '幢', '噸', '坨', '槍', '扎', '樣', '窠', '捆', '匹', '組', '枝', '群', '杆', '貼', '米', '桶', '摞', '檔', '筒', '路', '紙', '壺', '段', '瓶', '種', '刀', '臺', '匹', '伙', '冊', '泓', '幅', '疊', '股', '腔', '頂', '穴', '梭', '圈', '券', '捲')
pattern = r'((?:\d+|[零一二三四五六七八九十百千萬億])+)\s*(' + '|'.join(quantifiers) + r')'

# 自定義函數：產品萃取規格資訊
def extract_spec(name):
    # 使用正則表達式去匹配規格單位與數字
    matches = re.findall(pattern, name)
    if matches:
        # 將中文轉換成阿拉伯數字
        matches = [(chinese_to_arabic_number(match[0]), match[1]) for match in matches]
        return matches
    else:
        return None

# 自定義函數：中文數字轉換為阿拉伯數字
def chinese_to_arabic_number(text):
    chinese_to_arabic = {'零': '0', '一': '1', '二': '2', '三': '3', '四': '4', '五': '5', '六': '6', '七': '7', '八': '8', '九': '9','十':'10'}
    for chinese, arabic in chinese_to_arabic.items():
        text = text.replace(chinese, arabic)
    return text

# 自定義函數：從'name' 或 'packingSpec' 列中萃取規格資訊(因為部分產品’packingSpec‘為缺失值，產品規格資訊存在於’name‘中)
def extract_spec_from_name(row):
    if pd.isnull(row['packingSpec']):
        # 如果 'packingSpec' 欄位為缺失值，則從 'name' 欄位中提取規格資訊
        name = row['name']
        # 使用自定義函數 extract_spec 萃取規格資訊
        return extract_spec(name)
    elif not re.match(pattern, row['packingSpec']):
        # 如果 'packingSpec' 欄位不符合規定的格式，則從 'name' 欄位中萃取規格資訊
        name = row['name']
        # 使用自定義函數 extract_spec 萃取規格資訊
        return extract_spec(name)
    else:
        # 如果 'packingSpec' 欄位已經有值且符合規定的格式，則返回原有值
        return extract_spec(row['packingSpec'])



# 新增packingAmount跟doseUnit欄位

def packingAmount(row):
    packingAmount = 0  # 初始化 packingAmount
    doseUnit = ''      # 初始化 doseUnit
    other_packing = [] # 初始化 other_packing
    if row['packingSpec'] is not None and len(row['packingSpec']) == 1:
        packingAmount = row['packingSpec'][0][0]
        doseUnit = row['packingSpec'][0][1]
        return pd.Series({'packingAmount': packingAmount, 'doseUnit': doseUnit, 'other_packing': ''})

    elif row['packingSpec'] is not None and len(row['packingSpec']) > 1:
        other_packing = []  # 創建一個列表來存儲除了 '粒', '包', '條' 以外的所有元素
        for i in row['packingSpec']:
            if i[1] in ['粒','包','條'] :
                packingAmount = i[0]
                doseUnit = i[1]
            else:
                other_packing.append(i)  # 將除了 '粒', '包', '條' 以外的元素添加到列表中
        return pd.Series({'packingAmount': packingAmount, 'doseUnit': doseUnit, 'other_packing': other_packing})

    else:
        # 如果 packingSpec 是 None 或空列表，則返回預設值
        return pd.Series({'packingAmount': 0, 'doseUnit': '', 'other_packing': ''})






#萃取每日攝取量/服用方法

# 自定義函數：萃取每日攝取量/服用方法的資訊
def extract_suggestion_intake(row):
    # 確認 'description' 和 'summary' 列是否有 NaN 值，如果有則替換成 ''
    description = str(row['description']) if pd.notnull(row['description']) else ''
    summary = str(row['summary']) if pd.notnull(row['summary']) else ''
    # 合併 'description' 和 'summary' 列中的文本資訊(因為每日攝取量/服用方法可能位於該兩個欄位中)
    text = description + ' ' + summary

    # 使用正則表達式匹配每日攝取量/服用方法的資訊
    usage_pattern = r'(每日|每次|每天|每回)\s*([\d一二三四五六七八九十百千万]+)?\s*(粒|個|顆|枚|串|張|盒|片|罐|瓶|杯|碗|包|塊|支|棵|根|條|樽|袋|箱|蒲|副|疋|尾|匹|門|口|家|頭|位|把|台|部|棟|間|輛|架|輪|座|艘|列|車|線|號|階|層|篇|本|冊|輯|份|席|曲|卷|步|拍|聲|刻|次|回|場|波|梭|節|袖|堆|塊|團|箇|輩|雙|峰|對|班|旅|勺|齣|滴|招|朶|缽|帖|彎|攤|套|幢|噸|坨|槍|扎|樣|窠|捆|匹|組|枝|群|杆|貼|米|桶|摞|檔|筒|路|紙|壺|段|瓶|種|刀|臺|匹|伙|冊|泓|幅|疊|股|腔|頂|穴|梭|圈|券|捲)'


    matches = re.findall(usage_pattern, text)
    # 将匹配结果按原始顺序排列
    suggestion_intake = ', '.join([match[0] + (match[1] if match[1] else '') + match[2] for match in matches])
    return suggestion_intake if suggestion_intake else None



#萃取配方原料資訊

# 自定義函數：萃取欄位中原料或配方的資訊
def extract_ingredients(row):
    # 確認 'description' 和 'summary' 列是否有 NaN 值，如果有則替換成 ''
    description = str(row['description']) if pd.notnull(row['description']) else ''
    summary = str(row['summary']) if pd.notnull(row['summary']) else ''
    # 合併 'description' 和 'summary' 列中的文本資訊(因為配方原料資訊可能位於該兩個欄位中)
    text = description + ' ' + summary
    # 定義正則表達式之模式，匹配以"成分："或"含："或"原料："作為開頭的文本
    pattern = r'(?:成分|含|原料|含有|成份|含量資料)\s*(.*?)[。；\n]'

    # 使用正則表達式進行匹配
    matches = re.findall(pattern, text)

    # 返回匹配到的原料列表
    return ';'.join(matches) if matches else None

#萃取配方劑型資訊

# 自定義函數：萃取產品的劑型
def extract_dosage_form(row):
    text = str(row['description']).lower()

    if '丸' in text:
        return '丸'
    elif any(keyword in text for keyword in ['膠囊']):
        return '膠囊'
    elif any(keyword in text for keyword in ['錠','含片','片']):
        return '錠劑'
    elif any(keyword in text for keyword in ['飲', '液', '毫升', 'ml', '汁']):
        return '飲品'
    elif any(keyword in text for keyword in ['膠囊','粒']):
        return '膠囊'
    elif any(keyword in text for keyword in ['粉', '末']):
        return '粉末'
    else:
        return None





# 將這個函數應用於資料框的每一行
df['packingSpec'] = df.apply(extract_spec_from_name, axis=1)
df[['packingAmount', 'doseUnit','other_packing']] = df.apply(packingAmount, axis=1)
df['suggestionIntake'] = df.apply(extract_suggestion_intake, axis=1)
df['ingredients'] = df.apply(extract_ingredients, axis=1)
df['dosageForm(ref)'] = df.apply(extract_dosage_form, axis=1)





# 資料輸出儲存

In [4]:
# 儲存清洗完成的資料
trimed_file_name = '2024-05-17_hktvmall_brainHealthProduct_trimed'
df.to_csv(f'/content/drive/My Drive/Colab Notebooks/{trimed_file_name}.csv')

# 產品名稱關鍵字分析(詞頻矩陣)

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


# 將文本數據存儲在列表中，同步確認產品name欄位中沒有缺失值
lux = df.dropna(subset=['name'], inplace=False)
texts = lux['name'].tolist()


# 初始化 CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1))  # 設置 n-grams 固定為1個詞

# 將文本數據轉換為詞頻矩陣
X = vectorizer.fit_transform(texts)

# 取得特徵名稱（即 n-grams）
feature_names = vectorizer.get_feature_names_out()

# 計算每個 n-gram 的出現次數
ngrams_counts = X.sum(axis=0).A1

# 將特徵名稱和對應的出現次數合併成字典
ngrams_counts_dict = dict(zip(feature_names, ngrams_counts))

# 打印出現次數最多的前10個 n-grams
print(sorted(ngrams_counts_dict.items(), key=lambda x: x[1], reverse=True)[:10])

[('平行進口', 826), ('此日期前最佳', 397), ('毫克', 278), ('60粒', 271), ('nmn', 214), ('參考效期', 149), ('dha', 130), ('2025', 122), ('平行進口貨', 108), ('120粒', 100)]


# 關鍵字輸出並儲存


In [8]:

# 設定檔案名稱
save_file_name = '2024-05-13_hktvmall_大腦產品關鍵字'


# 使用 DataFrame 構造函數創建 DataFrame
df_key = pd.DataFrame(ngrams_counts_dict)

# 打印 DataFrame
df_sorted = df_key.sort_values(by='ngrams_counts', ascending=False)
df_sorted.to_csv(f'/content/drive/My Drive/Colab Notebooks/{save_file_name}.csv')
df_sorted

,feature_names,ngrams_counts
1986,平行進口,826
2461,此日期前最佳,397
2488,毫克,278
697,60粒,271
1128,nmn,214
...,...,...
1650,卡路士,1
1652,印加果油,1
1653,印度人蔘,1
1654,印度古方天然營養補充品,1
